In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

import random
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
rawData = pd.read_csv('Skype.csv')
rawData = rawData[~rawData.isin([np.nan,np.inf,-np.inf]).any(1)] # drop all inf and nan rows
# rawTest = pd.read_csv('facebook.csv')
# rawTest = rawTest[~rawTest.isin([np.nan,np.inf,-np.inf]).any(1)]

In [ ]:
X = np.array(rawData.loc[:,['Tot Fwd Pkts','Tot Bwd Pkts','TotLen Fwd Pkts','TotLen Bwd Pkts','Fwd Pkt Len Max','Bwd Pkt Len Max','FirstNPkt_size','Flow Duration','Flow Byts/s','Flow Pkts/s']])
Y = rawData.loc[:,['Label']]
Y = np.array(pd.get_dummies(Y,columns=['Label']))

In [ ]:
Xtrain,XTest,Ytrain,YTest = train_test_split(X,Y,test_size=0.3,random_state=21)

#Ytrain = tf.keras.utils.to_categorical(Ytrain, num_classes=3)
#YTest = tf.keras.utils.to_categorical(YTest, num_classes=3)
print(Ytrain.shape)
print(YTest.shape)
#Ytrain = Ytrain.reshape(Ytrain.shape[0],)
Ytrain = Ytrain.astype(np.float32)

#YTest = YTest.reshape(YTest.shape[0],)
YTest = YTest.astype(np.float32)
print("Training set size:", Xtrain.shape)
print("Test set size:",XTest.shape)

In [ ]:
n_x = X.shape[1]
n_y = Y.shape[1]
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(n_x, input_dim=10, activation='relu'))
    model.add(Dense(n_y, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=baseline_model,epochs=100, batch_size=5, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)

In [ ]:
results = cross_val_score(model, X, Y, cv=kfold)

In [ ]:
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
# test_loss, test_acc = model.evaluate(XTest,YTest,verbose=1)
# print('Test Accuracy:',test_acc)